In [77]:
import pandas as pd
import numpy as np

#machine learning modules
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Firstly, we read the files to obtain the information. 

In [78]:
df = pd.read_csv('train.csv') #train import
df_test = pd.read_csv('test.csv') #test import
df_test_submission = pd.read_csv('survival_submission.csv') #real values of survival
df #print the info

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [79]:
df.isnull().sum() #we check if are null values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

We delete the columns that are not intereseting: Name, SibSp, Parch, Ticket and Cabin. We set PassengerId as index to compare the results without affecting the model.

In [80]:
df.set_index(df['PassengerId'], inplace=True)
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], inplace=True)
df

,Survived,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,,
1,0,3,male,22.0,7.2500,S
2,1,1,female,38.0,71.2833,C
3,1,3,female,26.0,7.9250,S
4,1,1,female,35.0,53.1000,S
5,0,3,male,35.0,8.0500,S
...,...,...,...,...,...,...
887,0,2,male,27.0,13.0000,S
888,1,1,female,19.0,30.0000,S
889,0,3,female,NaN,23.4500,S


In [81]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
Embarked      2
dtype: int64

Now we check if the data we have left is clean. 

In [82]:
df.describe()

,Survived,Pclass,Age,Fare
count,891.000000,891.000000,714.000000,891.000000
mean,0.383838,2.308642,29.699118,32.204208
std,0.486592,0.836071,14.526497,49.693429
min,0.000000,1.000000,0.420000,0.000000
25%,0.000000,2.000000,20.125000,7.910400
50%,0.000000,3.000000,28.000000,14.454200
75%,1.000000,3.000000,38.000000,31.000000
max,1.000000,3.000000,80.000000,512.329200


We only have to deal with the age null values. We are going to set their value to their gender mean. 

In [83]:
age_means = df.groupby('Sex')['Age'].mean()
df['Age'] = df.apply(lambda row : age_means[row['Sex']] if np.isnan(row['Age']) else row['Age'], axis=1)

df.dropna(inplace=True) #we remove any other null value
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
dtype: int64

We create a function to implement all we have done later. 

In [84]:
def clean_dataset(df_aux):
    df = df_aux.copy()
    df.set_index(df['PassengerId'], inplace=True)
    df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], inplace=True)
    age_means = df.groupby('Sex')['Age'].mean()
    df['Age'] = df.apply(lambda row : age_means[row['Sex']] if np.isnan(row['Age']) else row['Age'], axis=1)

    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode())
    return df

We split the data into Categorical and Numerical. We also define new variables for the training model without using the main data. 

In [85]:
X_train = df.drop(columns=['Survived'])
y_train = df['Survived']
X_train_CAT = X_train[['Pclass', 'Sex', 'Embarked']] #categorical
X_train_NUM = X_train[['Age', 'Fare']] #numerical

For the Categorical data we use LabelEncoder, to transform the text into normalize data. 

In [86]:
p_class_encoder = LabelEncoder() #P_class manipulation
p_class_encoder.fit(X_train_CAT['Pclass'])
X_train['Pclass'] = p_class_encoder.transform(X_train_CAT['Pclass'])

sex_encoder = LabelEncoder() #Sex maniopulation
sex_encoder.fit(X_train_CAT['Sex'])
X_train['Sex'] = sex_encoder.transform(X_train_CAT['Sex'])

embarked_encoder = LabelEncoder() #Embarked manipulation
embarked_encoder.fit(X_train_CAT['Embarked'])
X_train['Embarked'] = embarked_encoder.transform(X_train_CAT['Embarked'])

X_train #modified

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1,2,1,22.000000,7.2500,2
2,0,0,38.000000,71.2833,0
3,2,0,26.000000,7.9250,2
4,0,0,35.000000,53.1000,2
5,2,1,35.000000,8.0500,2
...,...,...,...,...,...
887,1,1,27.000000,13.0000,2
888,0,0,19.000000,30.0000,2
889,2,0,27.915709,23.4500,2


For the Numerical data we use StandardScaler, to normalize the numerical values: $z = (x - \text{mean})/\text{std}$ 

In [87]:
age_scaler = StandardScaler() #age manipulation
age_scaler.fit(X_train_NUM[['Age']])
X_train['Age'] = age_scaler.transform(X_train_NUM[['Age']])

fare_scaler = StandardScaler() #fare manipulation
fare_scaler.fit(X_train_NUM[['Fare']])
X_train['Fare'] = fare_scaler.transform(X_train_NUM[['Fare']])

X_train #modified

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1,2,1,-0.592752,-0.500240,2
2,0,0,0.640471,0.788947,0
3,2,0,-0.284446,-0.486650,2
4,0,0,0.409241,0.422861,2
5,2,1,0.409241,-0.484133,2
...,...,...,...,...,...
887,1,1,-0.207370,-0.384475,2
888,0,0,-0.823981,-0.042213,2
889,2,0,-0.136790,-0.174084,2


Now we apply the same procedure to the test data. 

In [88]:
df_test = clean_dataset(df_test).copy()
X_test = df_test
y_test = df_test_submission['Survived']

X_test_CAT = X_test[['Pclass', 'Sex', 'Embarked']]
X_test_NUM = X_test[['Age', 'Fare']]

#Categorical
X_test['Pclass'] = p_class_encoder.transform(X_test_CAT['Pclass'])
X_test['Sex'] = sex_encoder.transform(X_test_CAT['Sex'])
X_test['Embarked'] = embarked_encoder.transform(X_test_CAT['Embarked'])

#Numerical
X_test['Age'] = age_scaler.transform(X_test_NUM[['Age']])
X_test['Fare'] = fare_scaler.transform(X_test_NUM[['Fare']])

X_test #modified test table

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
892,2,1,0.370703,-0.488579,1
893,2,0,1.334158,-0.505273,2
894,1,1,2.490305,-0.451165,1
895,2,1,-0.207370,-0.471802,2
896,2,0,-0.592752,-0.398819,2
...,...,...,...,...,...
1305,2,1,0.044880,-0.484133,2
1306,0,0,0.717547,1.546286,0
1307,2,1,0.679009,-0.500240,2


Lastly, we use a model to predict the chances of survival of the test sample. We train the model and then it makes a prediction. 

In [89]:
model = RandomForestClassifier()

model.fit(X_train, y_train) #we train the model
y_predicted = model.predict(X_test) #test prediction

accuracy_score(y_test, y_predicted) #model acuraccy

0.8133971291866029